In [1]:
import $ivy.`org.apache.spark::spark-sql:3.2.0`
import $ivy.`sh.almond::almond-spark:0.10.1`

import org.apache.spark._
import org.apache.spark.sql.{functions => func, _}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

import org.slf4j.LoggerFactory
import org.apache.log4j.{Level, Logger}

val spark = SparkSession
      .builder()
      .config("spark.sql.shuffle.partitions", 4)
      .master("local[4]")
      .getOrCreate()

import spark.implicits._

Logger.getRootLogger().setLevel(Level.ERROR)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
22/05/12 11:24:15 INFO SparkContext: Running Spark version 3.2.0
22/05/12 11:24:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/05/12 11:24:15 INFO ResourceUtils: ==============================================================
22/05/12 11:24:15 INFO ResourceUtils: No custom resources configured for spark.driver.
22/05/12 11:24:15 INFO ResourceUtils: ==============================================================
22/05/12 11:24:15 INFO SparkContext: Submitted application: 4b5db212-4396-4158-a2d0-968178fb5ef2
22/05/12 11:24:15 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, amount: 1.0)
22/05/1

import $ivy.$                                  

import $ivy.$                               


import org.apache.spark._

import org.apache.spark.sql.{functions => func, _}

import org.apache.spark.sql.functions._

import org.apache.spark.sql.types._


import org.slf4j.LoggerFactory

import org.apache.log4j.{Level, Logger}


spark: SparkSession = org.apache.spark.sql.SparkSession@784c0404
import spark.implicits._



In [2]:
val entiretable = spark.read.format("csv")
    .option("header", "true")
    .option("sep", ",")
    .load("../data/police/NYPD_Complaint_Data_Historic.csv")    
    .withColumn("CMPLNT_FR_DATETIME", to_timestamp(concat(col("CMPLNT_FR_DT"), lit(" "), col("CMPLNT_FR_TM")), "MM/dd/yyyy HH:mm:ss"))
    .withColumn("COMPLETE_LOC", concat(col("LOC_OF_OCCUR_DESC"), lit(" "), col("PREM_TYP_DESC")))

entiretable: DataFrame = [CMPLNT_NUM: string, CMPLNT_FR_DT: string ... 24 more fields]

In [3]:
val crimesManh = entiretable
    .withColumn("CMPLNT_FR_DATETIME_2H_AFTER", col("CMPLNT_FR_DATETIME") + expr("INTERVAL 2 HOURS"))
    .where((col("BORO_NM") === "MANHATTAN" || col("BORO_NM") === "BRONX") && year(col("CMPLNT_FR_DATETIME")) === 2015)
    .withColumnRenamed("CMPLNT_FR_DATETIME", "CMPLNT_FR_DATETIME_MANH")
    .withColumnRenamed("CMPLNT_FR_DATETIME_2H_AFTER", "CMPLNT_FR_DATETIME_2H_AFTER_MANH")
    .withColumnRenamed("CMPLNT_FR_DT", "CMPLNT_FR_DT_MANH")
    .withColumnRenamed("CMPLNT_NUM", "CMPLNT_NUM_MANH")
    .select("CMPLNT_FR_DATETIME_MANH", "CMPLNT_FR_DATETIME_2H_AFTER_MANH", "CMPLNT_FR_DT_MANH", "CMPLNT_NUM_MANH")
    

crimesManh: DataFrame = [CMPLNT_FR_DATETIME_MANH: timestamp, CMPLNT_FR_DATETIME_2H_AFTER_MANH: timestamp ... 2 more fields]

In [4]:
val crimesQue = entiretable
    .withColumn("CMPLNT_FR_DATETIME_2H_AFTER_QUE", col("CMPLNT_FR_DATETIME") + expr("INTERVAL 2 HOURS"))
    .where((col("BORO_NM") === "QUEENS" || col("BORO_NM") === "BROOKLYN") && year(col("CMPLNT_FR_DATETIME")) === 2015)
    .withColumnRenamed("CMPLNT_FR_DATETIME", "CMPLNT_FR_DATETIME_QUE")
    .withColumnRenamed("CMPLNT_FR_DATETIME_2H_AFTER", "CMPLNT_FR_DATETIME_2H_AFTER_QUE")
    .withColumnRenamed("CMPLNT_FR_DT", "CMPLNT_FR_DT_QUE")
    .withColumnRenamed("CMPLNT_NUM", "CMPLNT_NUM_QUE")
    .select("CMPLNT_FR_DATETIME_QUE", "CMPLNT_FR_DATETIME_2H_AFTER_QUE", "CMPLNT_FR_DT_QUE", "CMPLNT_NUM_QUE")

crimesQue: DataFrame = [CMPLNT_FR_DATETIME_QUE: timestamp, CMPLNT_FR_DATETIME_2H_AFTER_QUE: timestamp ... 2 more fields]

In [4]:
//crimesQue.count()

In [5]:
val results = crimesManh
    .join(crimesQue, crimesManh.col("CMPLNT_FR_DT_MANH") === crimesQue.col("CMPLNT_FR_DT_QUE"), "fullouter")
    .where(col("CMPLNT_FR_DATETIME_QUE") <= col("CMPLNT_FR_DATETIME_2H_AFTER_MANH") &&
               col("CMPLNT_FR_DATETIME_QUE") >= col("CMPLNT_FR_DATETIME_MANH"))
    .select("CMPLNT_FR_DATETIME_MANH", "CMPLNT_NUM_MANH", "CMPLNT_FR_DATETIME_QUE", "CMPLNT_NUM_QUE")

results: DataFrame = [CMPLNT_FR_DATETIME_MANH: timestamp, CMPLNT_NUM_MANH: string ... 2 more fields]

In [6]:
results.write.mode("overwrite").parquet("../out/police.parquet")